<a href="https://colab.research.google.com/github/naya18g/Project/blob/main/Devnagri.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.transform import resize
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, BatchNormalization
from keras.utils import np_utils
from sklearn.metrics import accuracy_score

In [ ]:
target_train = []
img_train = []
img_train_flat = []
DATADIR = '/kaggle/input/devnagri-script-classification/Data/Train'
targets = os.listdir(DATADIR)
len(targets)

In [ ]:
for target in targets:
    path = os.path.join(DATADIR,target)
    label  = targets.index(target)
    for img in os.listdir(path):
        img_array = imread(os.path.join(path,img))
        img_train.append(img_array)
        img_train_flat.append(img_array.flatten())
        target_train.append(label)
img_train_flat = np.array(img_train_flat) 
target_train = np.array(target_train)
img_train = np.array(img_train)
        

In [ ]:
img_train.shape

In [ ]:
img_train_flat = img_train_flat / 255.0

In [ ]:
id_test = []
for i in range(1,13801):
    id_test.append(str(i)+".png")

In [ ]:
DATADIR_test = '/kaggle/input/devnagri-script-classification/Data/Test'
img_test = []
img_test_flat = []
for img in id_test:
    img_array = imread(os.path.join(DATADIR_test,img))
    img_test.append(img_array)
    img_test_flat.append(img_array.flatten())
img_test_flat = np.array(img_test_flat)
img_test = np.array(img_test)

In [ ]:
img_test_flat = img_test_flat / 255.0

In [ ]:
img_train_flat

In [ ]:
img_train

In [ ]:
img_test_flat

In [ ]:
img_array

In [ ]:
img_array.shape

In [ ]:
img_test_flat.shape

In [ ]:
img_train_flat.shape

In [ ]:
img_train.shape

In [ ]:
img_test.shape

In [ ]:
x = img_train
y = target_train


In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
x_train.shape

In [ ]:
x_train = x_train.reshape(-1,32,32,1)

In [ ]:
x_train.shape

In [ ]:
y_train.shape

In [ ]:
x_test.shape

In [ ]:
x_test = x_test.reshape(-1,32,32,1)

In [ ]:
x_test.shape

In [ ]:
y_test.shape

In [ ]:
y_train  = np_utils.to_categorical(y_train,46)
y_train.shape

In [ ]:
y_test  = np_utils.to_categorical(y_test,46)
y_test.shape

In [ ]:
model = Sequential()
model.add(Conv2D(40, kernel_size = 5,  activation='relu', input_shape = (32,32,1)))
model.add(BatchNormalization())
model.add(Conv2D(40,kernel_size = 3 ,strides=2, padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(Dense(60, activation='relu'))
model.add(BatchNormalization())
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(120, activation='relu'))
model.add(Dense(46, activation='softmax'))


model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

In [ ]:
training_history = model.fit(
x_train,
y_train, 
batch_size=32,
verbose=1, 
epochs=10,
validation_data=(x_test, y_test)
)

In [ ]:
img_train = img_train.reshape(-1,32,32,1)
img_test = img_test.reshape(-1,32,32,1)
pred = model.predict(img_test)
  
    

In [ ]:
y_preds = []
for i in range(0,13800):
    y_preds.append(targets[np.argmax(pred[i])])

In [ ]:
fid = pd.Series(id_test,name = "Id")
fpreds = pd.Series(y_preds,name="Category")
fid = pd.DataFrame(fid)
res = fid.join(fpreds)

In [ ]:
res.to_csv("submission1.csv",index=False)